In [25]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/journal_synthetic.csv")

# Feature engineering
df["GrossPerTaxable"] = df["Gross"] / (df["Taxable"].replace(0, np.nan))
df["IsReclass"] = (df["InputSource"] == "reclassin").astype(int)
df["IsBalanceImport"] = (df["InputSource"] == "balanceimport").astype(int)
df["IsReclassOut"] = (df["InputSource"] == "reclassout").astype(int)
df["MonthName"] = df["Month"].map({1:"Jan",2:"Feb",3:"Mar",4:"Apr",5:"May",6:"Jun",7:"Jul",8:"Aug",9:"Sep",10:"Oct",11:"Nov",12:"Dec"})
df["YearMonth"] = df["Year"].astype(str) + "-" + df["Month"].astype(str).str.zfill(2)
df["CityCounty"] = df["City"] + "_" + df["County"]
df["LogGross"] = np.log1p(df["Gross"])
df["LogTaxable"] = np.log1p(df["Taxable"].abs())

df.head()

c:\Repos\FinalYearProject\IndirectTax\IndirectTax\ML\venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Region,City,County,Entity,District,Gross,Taxable,InputSource,PlaceDetermination,EntryId,...,Month,GrossPerTaxable,IsReclass,IsBalanceImport,IsReclassOut,MonthName,YearMonth,CityCounty,LogGross,LogTaxable
0,US-FL,Boca Raton,Palm Beach,PizzaHut,District-50,5192.56,363.48,balanceimport,taxareacode,100000,...,3,14.285683,0,1,0,Mar,2023-03,Boca Raton_Palm Beach,8.555175,5.898472
1,US-MI,Troy,Oakland,PizzaHut,District-13,5632.31,464.67,balanceimport,taxareacode,100001,...,12,12.121097,0,1,0,Dec,2022-12,Troy_Oakland,8.636452,6.143477
2,US-NC,Charlotte,Mecklenburg,Reliance,District-27,6192.36,789.53,balanceimport,taxareacode,100002,...,5,7.843097,0,1,0,May,2024-05,Charlotte_Mecklenburg,8.731233,6.672704
3,US-NY,Manhattan,New York,PizzaHut,District-43,5284.76,418.02,balanceimport,taxareacode,100003,...,1,12.642362,0,1,0,Jan,2023-01,Manhattan_New York,8.572772,6.037919
4,US-GA,Smyrna,Cobb,Reliance,District-35,8228.46,1209.58,balanceimport,taxareacode,100004,...,12,6.802741,0,1,0,Dec,2023-12,Smyrna_Cobb,9.015476,7.098855


In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder



In [27]:
# Prepare training data: only use rows where PlaceDetermination == "reclass"
train_df = df[df["PlaceDetermination"] == "reclass"].copy()



In [28]:
# Encode categorical features with persistent encoders
cat_cols = ["Region", "County", "Entity", "District", "MonthName", "YearMonth", "CityCounty"]
encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col].astype(str))
    encoders[col] = le

# Target encoding
city_le = LabelEncoder()
train_df["CityEncoded"] = city_le.fit_transform(train_df["City"].astype(str))



In [ ]:
# Features for training
features = ["Region", "County", "Entity", "District", "Gross", "Taxable", "TaxRate", "Year", "Month", "GrossPerTaxable", "IsReclass", "IsBalanceImport", "IsReclassOut", "MonthName", "YearMonth", "CityCounty", "LogGross", "LogTaxable"]
X_train = train_df[features]
y_train = train_df["CityEncoded"]

# Train model
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)

# --- TESTING ---

# Read test data
test_df = pd.read_csv("../data/reconcillation_synthetic.csv")

# Only test on rows with UnreportedTax > 0
test_df = test_df[test_df["UnreportedTax"] > 0].copy()

# For each test row, find all possible reclass rows from training data with the same Region and County
# Prepare test features (encode using training encoders)
for col in ["Region", "County", "Entity", "District"]:
    if col not in test_df.columns:
        test_df[col] = -1  # Fill missing columns with -1
    le = encoders[col]
    test_df[col] = test_df[col].astype(str).map(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
test_df["MonthName"] = test_df["Month"].map({1:"Jan",2:"Feb",3:"Mar",4:"Apr",5:"May",6:"Jun",7:"Jul",8:"Aug",9:"Sep",10:"Oct",11:"Nov",12:"Dec"})
test_df["YearMonth"] = test_df["Year"].astype(str) + "-" + test_df["Month"].astype(str).str.zfill(2)
test_df["CityCounty"] = test_df["City"] + "_" + test_df["County"].astype(str)
for col in ["MonthName", "YearMonth", "CityCounty"]:
    le = encoders[col]
    test_df[col] = test_df[col].astype(str).map(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
test_df["LogGross"] = np.log1p(test_df["Gross"])
test_df["LogTaxable"] = np.log1p(test_df["Taxable"].abs())
test_df["GrossPerTaxable"] = test_df["Gross"] / (test_df["Taxable"].replace(0, np.nan))
test_df["IsReclass"] = 0
test_df["IsBalanceImport"] = 0
test_df["IsReclassOut"] = 0

# Predict city for each test row, restrict to same region and county (with debug)
predicted_cities = []
for idx, row in test_df.iterrows():
    region_val = row["Region"]
    county_val = row["County"]
    # Find all possible city encodings from the same region and county in the training data
    possible_cities = train_df[(train_df["Region"] == region_val) & (train_df["County"] == county_val)]["CityEncoded"].unique()
    if len(possible_cities) == 0:
        # Debug: print when fallback happens
        print(f"No city found for region={region_val}, county={county_val} in train. Fallback to all cities.")
        possible_cities = train_df["CityEncoded"].unique()
    # Use DataFrame to preserve feature names
    row_df = pd.DataFrame([row[features]], columns=features)
    probs = clf.predict_proba(row_df)[0]
    mask = np.zeros_like(probs, dtype=bool)
    mask[possible_cities] = True
    probs_masked = probs * mask
    if probs_masked.sum() == 0:
        # Debug: print when fallback happens
        print(f"All masked out for region={region_val}, county={county_val}. Fallback to original prediction.")
        pred = np.argmax(probs)
    else:
        pred = np.argmax(probs_masked)
    predicted_cities.append(pred)
test_df["PredictedCity"] = city_le.inverse_transform(predicted_cities)

# Debug: Check if predicted city is in the same region and county as test row
check = []
for idx, row in test_df.iterrows():
    pred_city = row["PredictedCity"]
    region_val = row["Region"]
    county_val = row["County"]
    # Find all cities in train for this region and county
    valid_cities = train_df[(train_df["Region"] == region_val) & (train_df["County"] == county_val)]["City"].unique()
    if pred_city not in valid_cities:
        check.append((region_val, county_val, row["City"], pred_city))
if check:
    print("Predicted city not in same region/county for these rows:")
    for item in check:
        print(f"Region: {item[0]}, County: {item[1]}, True City: {item[2]}, Predicted: {item[3]}")
else:
    print("All predicted cities are from the same region and county as the test row.")

# Debug: Check all rows, not just head()
output_df_full = test_df.copy()
print(output_df_full)

# Also, print all mismatches if any
if check:
    print("Predicted city not in same region/county for these rows (full list):")
    for item in check:
        print(f"Region: {item[0]}, County: {item[1]}, True City: {item[2]}, Predicted: {item[3]}")
else:
    print("All predicted cities are from the same region and county as the test row.")

All masked out for region=3, county=6. Fallback to original prediction.
Predicted city not in same region/county for these rows:
Region: 3, County: 6, True City: Wheaton, Predicted: Astoria
